# PartGraph 改进版训练

## 主要改进
1. **多任务学习**: 分类损失 + 重建损失 (分类为主)
2. **增加容量**: num_slots=8, slot_dim=256
3. **多样性损失**: 防止 slot collapse
4. **改进初始化**: 每个 slot 独立初始化

In [ ]:
# 1. 检查 GPU
!nvidia-smi

In [ ]:
# 2. 克隆/更新代码
!git clone https://github.com/alltobebetter/PartGraph-FewShot.git 2>/dev/null || (cd PartGraph-FewShot && git pull)
%cd PartGraph-FewShot

In [ ]:
# 3. 安装依赖
!pip install -q -r requirements.txt

In [ ]:
# 4. 下载 CUB-200 数据集
!mkdir -p data
%cd data
!wget -q -N https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz
!tar -xzf CUB_200_2011.tgz 2>/dev/null || echo 'Already extracted'
%cd ..

In [ ]:
# 5. 运行改进版训练
!python src/train_improved.py \
    --data_root ./data \
    --output_dir ./checkpoints_improved \
    --batch_size 32 \
    --epochs 30 \
    --num_slots 8 \
    --slot_dim 256 \
    --lambda_cls 1.0 \
    --lambda_recon 0.1 \
    --lambda_div 0.01

In [ ]:
# 6. Few-Shot 评估
!python src/eval_fewshot.py \
    --data_root ./data \
    --checkpoint ./checkpoints_improved/best_model.pth \
    --num_slots 8 \
    --slot_dim 256 \
    --n_way 5 \
    --k_shot 5 \
    --n_episodes 100

In [ ]:
# 7. 查看可视化结果
from IPython.display import Image, display
import glob

vis_files = sorted(glob.glob('./checkpoints_improved/epoch_*_vis.png'))
for f in vis_files[-5:]:  # 显示最后5个
    print(f)
    display(Image(filename=f))